## Section A : Initialization

### 1. Sample Data Restoration

![Loading data in sqliteonline](Images/A-1.png)

#### Loading Data in this Notebook

In [1]:
#!pip install sqlalchemy==1.3.9
#!pip install ipython-sql
%load_ext sql

In [2]:
%sql sqlite:///queue_system_sqlite3.db

In [3]:
%%sql
SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///queue_system_sqlite3.db
Done.


name
call_statuses
sqlite_sequence
calls
counters
jobs
languages
permissions
queues
role_has_permissions
roles


### 2. Table definition and their relation

#### Tables Definition:

1. **call_statuses**
   - `id` (integer) - NOT NULL, PRIMARY KEY, AUTOINCREMENT
   - `name` (varchar(191)) - NOT NULL; Status of the call i.e. `served` or `no show`.
   - `created_at` (timestamp) - DEFAULT NULL; Timestamp when the entry was created
   - `updated_at` (timestamp) - DEFAULT NULL; Timestamp when the entry was last updated
	
2. **calls**
   - `id` (integer) - NOT NULL, PRIMARY KEY, AUTOINCREMENT
   - `queue_id` (integer) - NOT NULL; Foreign key referencing `queues(id)`
   - `service_id` (integer) - NOT NULL; Foreign key referencing `services(id)`
   - `counter_id` (integer) - NOT NULL; Foreign key referencing `counters(id)`
   - `user_id` (integer) - NOT NULL; Foreign key referencing `users(id)`
   - `token_letter` (varchar(191)) - NOT NULL; Type of service i.e. `W`, `NR`, or `T`.
   - `token_number` (integer) - NOT NULL; Assigned token number for the call
   - `called_date` (date) - NOT NULL; Date when call was made
   - `started_at` (datetime) - DEFAULT current_timestamp; the data and time when the call started
   - `ended_at` (datetime) - DEFAULT NULL; the data and time when the call started
   - `waiting_time` (time) - DEFAULT NULL; period of time the caller waited
   - `served_time` (time) - DEFAULT NULL; period of time the call was answered for
   - `turn_around_time` (time) - DEFAULT NULL; total of waiting and served time
   - `created_at` (timestamp) - DEFAULT NULL; Timestamp when the entry was created
   - `updated_at` (timestamp) - DEFAULT NULL; Timestamp when the entry was last updated
   - `call_status_id` (integer) - DEFAULT NULL; Foreign key referencing `call_statuses(id)`

3. **counters**
   - `id` (integer) - NOT NULL, PRIMARY KEY, AUTOINCREMENT
   - `name` (varchar(191)) - NOT NULL; Type of `Counter 1` to `Counter 10`
   - `status` (integer) - NOT NULL; Contains only one value i.e. `1`
   - `created_at` (timestamp) - DEFAULT NULL; Timestamp when the entry was created
   - `updated_at` (timestamp) - DEFAULT NULL; Timestamp when the entry was last updated

  
4. **jobs**: Empty table
   - `id` (integer) - NOT NULL, PRIMARY KEY, AUTOINCREMENT
   - `queue` (varchar(191)) - NOT NULL
   - `payload` (longtext) - NOT NULL
   - `attempts` (integer) -  NOT NULL
   - `reserved_at` (integer) - DEFAULT NULL
   - `available_at` (integer) - NOT NULL
   - `created_at` (integer) - NOT NULL

5. **languages**
   - `id` (integer) - NOT NULL, PRIMARY KEY, AUTOINCREMENT
   - `code` (varchar(191)) - NOT NULL, UNIQUE; Identification code for each language
   - `name` (varchar(191)) - NOT NULL; Name of the language
   - `display` (varchar(191)) - NOT NULL; Language name + gender (Except Arabic all others are female)
   - `image` (varchar(191)) - DEFAULT NULL
   - `created_at` (timestamp) - DEFAULT NULL; Timestamp when the entry was created
   - `updated_at` (timestamp) - DEFAULT NULL; Timestamp when the entry was last updated
   - `token_translation` (varchar(191)) - DEFAULT NULL; The word `Token` in various languages
   - `please_proceed_to_translation` (varchar(191)) - DEFAULT NULL; The phrase `Please proceed to` in various languages

6. **permissions**
   - `id` (integer) - NOT NULL, PRIMARY KEY, AUTOINCREMENT
   - `name` (varchar(191)) - NOT NULL, UNIQUE; Various type of actions which require permission
   - `guard_name` (varchar(191)) - NOT NULL, UNIQUE; Contains only one value i.e. `web`
   - `created_at` (timestamp) - DEFAULT NULL; Timestamp when the entry was created
   - `updated_at` (timestamp) - DEFAULT NULL; Timestamp when the entry was last updated

7. **queues**
   - `id` (integer) - NOT NULL, PRIMARY KEY, AUTOINCREMENT
   - `service_id` (integer) - NOT NULL; Foreign Key referencing `services(id)`
   - `number` (integer) - NOT NULL; Queue number
   - `called` (integer) - NOT NULL; Indicator if the queue has been called
   - `reference_no` (varchar(191)) - NOT NULL; Reference number for the queue
   - `letter` (varchar(191)) - NOT NULL; Letter associated with the queue
   - `name` (varchar(191)) - DEFAULT NULL; Name of the person in the queue
   - `email` (varchar(191)) - DEFAULT NULL; Email of the person in the queue
   - `phone` (varchar(191)) - DEFAULT NULL; Phone number of the person in the queue
   - `position` (integer) - DEFAULT NULL; Position in the queue
   - `created_at` (timestamp) - DEFAULT NULL; Timestamp when the entry was created
   - `updated_at` (timestamp) - DEFAULT NULL; Timestamp when the entry was last updated

8.  **role_has_permissions**
    - `permission_id` (integer) - NOT NULL, PRIMARY KEY; Foreign Key referencing `permissions(id)`
    - `role_id` (integer) - NOT NULL, PRIMARY KEY; Foreign Key referencing `roles(id)` which will be updated on delete cascade

9.  **roles**
    - `id` (integer) - NOT NULL, PRIMARY KEY, AUTOINCREMENT
    - `name` (varchar(191)) - NOT NULL, UNIQUE; Various types of roles i.e. `Admin`, `Agent`, `Super-Admin` or `Token`.
    - `guard_name` (varchar(191)) - NOT NULL, UNIQUE; Contains only one value i.e. `web`
    - `created_at` (timestamp) - DEFAULT NULL; Timestamp when the entry was created
    - `updated_at` (timestamp) - DEFAULT NULL; Timestamp when the entry was last updated

10. **services**
    - `id` (integer) - NOT NULL, PRIMARY KEY, AUTOINCREMENT
    - `name` (varchar(191)) - NOT NULL; Types of Services i.e. `WIS (W)`, `NEW REGISTRATION (NR)` and `TRAINING (T)`
    - `letter` (varchar(191)) - NOT NULL; Short hand form of each type of services i.e. `W`, `NR` and `T`.
    - `start_number` (integer) - NOT NULL; Contains only one value i.e. `1`
    - `status` (integer) - NOT NULL; Contains only one value i.e. `1`
    - `sms_enabled` (integer) - NOT NULL, DEFAULT 0; Contains only one value i.e. `0`
    - `optin_message_enabled` (integer) - NOT NULL, DEFAULT 0; Contains only one value i.e. `0`
    - `call_message_enabled` (integer) - NOT NULL, DEFAULT 0; Contains only one value i.e. `0`
    - `noshow_message_enabled` (integer) - NOT NULL, DEFAULT 0; Contains only one value i.e. `0`
    - `completed_message_enabled` (integer) - NOT NULL, DEFAULT 0; Contains only one value i.e. `0`
    - `status_message_enabled` (integer) - NOT NULL, DEFAULT 0; Contains only one value i.e. `0`
    - `optin_message_format` (varchar(191)) - DEFAULT NULL; Contains only one value i.e. `None`
    - `call_message_format` (varchar(191)) - DEFAULT NULL; Contains only one value i.e. `None`
    - `noshow_message_format` (varchar(191)) - DEFAULT NULL; Contains only one value i.e. `None`
    - `completed_message_format` (varchar(191)) - DEFAULT NULL; Contains only one value i.e. `None`
    - `status_message_format` (varchar(191)) - DEFAULT NULL; Contains only one value i.e. `None`
    - `status_message_positions` (varchar(191)) - DEFAULT NULL; Contains only one value i.e. `None`
    - `ask_name` (integer) - NOT NULL, DEFAULT 0; Contains only one value i.e. `0`
    - `name_required` (integer) - NOT NULL, DEFAULT 0; Contains only one value i.e. `0`
    - `ask_email` (integer) - NOT NULL, DEFAULT 0; Contains only one value i.e. `0`
    - `email_required` (integer) - NOT NULL, DEFAULT 0; Contains only one value i.e. `0`
    - `ask_phone` (integer) - NOT NULL, DEFAULT 0; Contains only one value i.e. `0`
    - `phone_required` (integer) - NOT NULL, DEFAULT 0; Contains only one value i.e. `0`
    - `created_at` (timestamp) - DEFAULT NULL; Timestamp when the entry was created
    - `updated_at` (timestamp) - DEFAULT NULL; Timestamp when the entry was last updated

11. **sqlite_sequence**
    - `name` - DEFAULT NULL
    - `seq` - DEFAULT NULL

12. **users**
    - `id` (integer) - NOT NULL, PRIMARY KEY, AUTOINCREMENT
    - `name` (varchar(191)) - NOT NULL; Name of systems user
    - `email` (varchar(191)) - NOT NULL, UNIQUE; Email of systems user
    - `email_verified_at` timestamp DEFAULT NULL
    - `password` (varchar(191)) - NOT NULL; Encrypted password
    - `remember_token` (varchar(100)) - DEFAULT NULL
    - `created_at` (timestamp) - DEFAULT NULL; Timestamp when the entry was created
    - `updated_at` (timestamp) - DEFAULT NULL; Timestamp when the entry was last updated
    - `image` (varchar(191)) - DEFAULT NULL; Path to image file

#### Tables Relationships:

1. **calls & queues (Many to 1 relation) :** Linked by `queue_id` in the `calls` table referencing `queues(id)`.
2. **calls & services (Many to 1 relation) :** Linked by `service_id` in the `calls` table referencing `services(id)`.
3. **calls & counters (Many to 1 relation) :** Linked by `counter_id` in the `calls` table referencing `counters(id)`.
4. **calls & users (Many to 1 relation) :** Linked by `user_id` in the `calls` table referencing `users(id)`.
5. **calls & call_statuses (Many to 0/1 relation) :** Linked by `call_status_id` in the `calls` table referencing `call_statuses(id)`.
6. **queues & services (Many to 1 relation) :** Linked by `service_id` in the `queues` table referencing `services(id)`.
7. **roles & permissions (1 to 1 relation) :** Linked by `role_id` and `permission_id` in the `role_has_permissions` table referencing `roles(id)` and `permissions(id)`, respectively.

#### SQL Query:

In [4]:
%%sql
SELECT name, sql FROM sqlite_master WHERE type = 'table' ORDER BY name;

 * sqlite:///queue_system_sqlite3.db
Done.


name,sql
call_statuses,"CREATE TABLE `call_statuses` ( `id` integer NOT NULL PRIMARY KEY AUTOINCREMENT, `name` varchar(191) NOT NULL, `created_at` timestamp NULL DEFAULT NULL, `updated_at` timestamp NULL DEFAULT NULL)"
calls,"CREATE TABLE `calls` ( `id` integer NOT NULL PRIMARY KEY AUTOINCREMENT, `queue_id` integer NOT NULL, `service_id` integer NOT NULL, `counter_id` integer NOT NULL, `user_id` integer NOT NULL, `token_letter` varchar(191) NOT NULL, `token_number` integer NOT NULL, `called_date` date NOT NULL, `started_at` datetime DEFAULT current_timestamp, `ended_at` datetime DEFAULT NULL, `waiting_time` time DEFAULT NULL, `served_time` time DEFAULT NULL, `turn_around_time` time DEFAULT NULL, `created_at` timestamp NULL DEFAULT NULL, `updated_at` timestamp NULL DEFAULT NULL, `call_status_id` integer DEFAULT NULL, CONSTRAINT `calls_call_status_id_foreign` FOREIGN KEY (`call_status_id`) REFERENCES `call_statuses` (`id`), CONSTRAINT `calls_counter_id_foreign` FOREIGN KEY (`counter_id`) REFERENCES `counters` (`id`), CONSTRAINT `calls_queue_id_foreign` FOREIGN KEY (`queue_id`) REFERENCES `queues` (`id`), CONSTRAINT `calls_service_id_foreign` FOREIGN KEY (`service_id`) REFERENCES `services` (`id`), CONSTRAINT `calls_user_id_foreign` FOREIGN KEY (`user_id`) REFERENCES `users` (`id`))"
counters,"CREATE TABLE `counters` ( `id` integer NOT NULL PRIMARY KEY AUTOINCREMENT, `name` varchar(191) NOT NULL, `status` integer NOT NULL, `created_at` timestamp NULL DEFAULT NULL, `updated_at` timestamp NULL DEFAULT NULL)"
jobs,"CREATE TABLE `jobs` ( `id` integer NOT NULL PRIMARY KEY AUTOINCREMENT, `queue` varchar(191) NOT NULL, `payload` longtext NOT NULL, `attempts` integer NOT NULL, `reserved_at` integer DEFAULT NULL, `available_at` integer NOT NULL, `created_at` integer NOT NULL)"
languages,"CREATE TABLE `languages` ( `id` integer NOT NULL PRIMARY KEY AUTOINCREMENT, `code` varchar(191) NOT NULL, `name` varchar(191) NOT NULL, `display` varchar(191) NOT NULL, `image` varchar(191) DEFAULT NULL, `created_at` timestamp NULL DEFAULT NULL, `updated_at` timestamp NULL DEFAULT NULL, `token_translation` varchar(191) DEFAULT NULL, `please_proceed_to_translation` varchar(191) DEFAULT NULL, UNIQUE (`code`))"
permissions,"CREATE TABLE `permissions` ( `id` integer NOT NULL PRIMARY KEY AUTOINCREMENT, `name` varchar(191) NOT NULL, `guard_name` varchar(191) NOT NULL, `created_at` timestamp NULL DEFAULT NULL, `updated_at` timestamp NULL DEFAULT NULL, UNIQUE (`name`,`guard_name`))"
queues,"CREATE TABLE `queues` ( `id` integer NOT NULL PRIMARY KEY AUTOINCREMENT, `service_id` integer NOT NULL, `number` integer NOT NULL, `called` integer NOT NULL, `reference_no` varchar(191) NOT NULL, `letter` varchar(191) NOT NULL, `name` varchar(191) DEFAULT NULL, `email` varchar(191) DEFAULT NULL, `phone` varchar(191) DEFAULT NULL, `position` integer DEFAULT NULL, `created_at` timestamp NULL DEFAULT NULL, `updated_at` timestamp NULL DEFAULT NULL, CONSTRAINT `queues_service_id_foreign` FOREIGN KEY (`service_id`) REFERENCES `services` (`id`))"
role_has_permissions,"CREATE TABLE `role_has_permissions` ( `permission_id` integer NOT NULL, `role_id` integer NOT NULL, PRIMARY KEY (`permission_id`,`role_id`), CONSTRAINT `role_has_permissions_permission_id_foreign` FOREIGN KEY (`permission_id`) REFERENCES `permissions` (`id`) ON DELETE CASCADE, CONSTRAINT `role_has_permissions_role_id_foreign` FOREIGN KEY (`role_id`) REFERENCES `roles` (`id`) ON DELETE CASCADE)"
roles,"CREATE TABLE `roles` ( `id` integer NOT NULL PRIMARY KEY AUTOINCREMENT, `name` varchar(191) NOT NULL, `guard_name` varchar(191) NOT NULL, `created_at` timestamp NULL DEFAULT NULL, `updated_at` timestamp NULL DEFAULT NULL, UNIQUE (`name`,`guard_name`))"
services,"CREATE TABLE `services` ( `id` integer NOT NULL PRIMARY KEY AUTOINCREMENT, `name` varchar(191) NOT NULL, `letter` varchar(191) NOT NULL, `start_number` integer NOT NULL, `status` integer NOT NULL, `sms_enabled` integer NOT NULL DEFAULT 0, `optin_message_enabled` in

In [5]:
%%sql
SELECT * FROM services

 * sqlite:///queue_system_sqlite3.db
Done.


id,name,letter,start_number,status,sms_enabled,optin_message_enabled,call_message_enabled,noshow_message_enabled,completed_message_enabled,status_message_enabled,optin_message_format,call_message_format,noshow_message_format,completed_message_format,status_message_format,status_message_positions,ask_name,name_required,ask_email,email_required,ask_phone,phone_required,created_at,updated_at
1,WIS (W),W,1,1,0,0,0,0,0,0,None,None,None,None,None,None,0,0,0,0,0,0,2024-01-23 10:29:32,2024-01-23 10:29:32
2,NEW REGISTRATION (NR),NR,1,1,0,0,0,0,0,0,None,None,None,None,None,None,0,0,0,0,0,0,2024-01-23 10:30:10,2024-01-23 10:34:26
3,TRAINING (T),T,1,1,0,0,0,0,0,0,None,None,None,None,None,None,0,0,0,0,0,0,2024-01-23 10:31:10,2024-01-23 10:31:10
